In [1]:
import os
os.environ["NUMBA_DISABLE_JIT"] = "1"

In [2]:
import sys, os
if 'google.colab' in sys.modules:
    %cd
    % rm -rf MPyDATA
    ! git clone --recurse-submodules -j8 https://github.com/Michaeldz36/MPyDATA.git
    %cd MPyDATA
    ! git checkout develop
    ! pip install -U $(cat requirements.txt | cut -d '=' -f 1)
else:
    sys.path.append(os.path.join(os.getcwd(), '../..'))

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from MPyDATA_examples.Olesik_et_al_2020.setup import Setup
from MPyDATA_examples.Olesik_et_al_2020.coordinates import x_id, x_p2, x_p3, x_log_of_pn
from MPyDATA.options import Options
from MPyDATA_examples.Olesik_et_al_2020.simulation import Simulation
from MPyDATA_examples.utils.show_plot import show_plot
from joblib import Parallel, parallel_backend, delayed
from MPyDATA_examples.Olesik_et_al_2020.physics.equilibrium_drop_growth import PdfEvolver
from MPyDATA.arakawa_c.discretisation import discretised_analytical_solution
from MPyDATA_examples.utils.error_norms import L2, Smolarkiewicz_Grabowski_1990_eq21
from MPyDATA_examples.Olesik_et_al_2020.convergence_plotter  import polar_plot

In [4]:
GCs = np.linspace(.05,.95, 5)
nrs = np.linspace(16, 1024, 5, dtype = int)

In [5]:
grid_layout = x_p2()
psi_coord = x_p2()
setup = Setup()

In [6]:
def analysis(setup, GC, opt):
    options = Options(**opt)
    simulation = Simulation(setup, grid_layout,  psi_coord, options, GC)
    simulation.step(simulation.out_steps[-1])
    t = simulation.out_steps[-1] * simulation.dt
    rh = simulation.rh
    pdf_t = PdfEvolver(setup.pdf, setup.drdt, t)
    analytical = discretised_analytical_solution(
                simulation.rh.magnitude,
                lambda r: pdf_t(r * rh.units).magnitude
            ) * pdf_t(rh[0]).units
    numerical = simulation.n
    
    maximum_numeric = np.max(numerical)
    maximum_analytic = np.max(analytical)
    measure_height = (maximum_numeric / maximum_analytic).magnitude
    
    #TODO: proportion of rel dispersion
    
    error = np.log2(Smolarkiewicz_Grabowski_1990_eq21(numerical.magnitude, analytical.magnitude, t.magnitude))
    return setup.nr, GC, error, measure_height

In [7]:
opt_set = (
     {'n_iters': 1},
     {'n_iters': 2},
     {'n_iters': 2,'infinite_gauge':True},
     {'n_iters': 2, 'infinite_gauge': True, 'flux_corrected_transport': True},
     {'n_iters': 2, 'third_order_terms':True},
     {'n_iters': 3},
     {'n_iters': 3, 'third_order_terms': True, 'infinite_gauge': True, 'flux_corrected_transport': True}
)


In [ ]:
for opt in opt_set:
    with parallel_backend('threading', n_jobs=-2):
        results0 = Parallel(verbose=10)(
            delayed(analysis)(Setup(nr = nr, mixing_ratios_g_kg = [1.1,]), GC, opt)
            for nr in nrs
            for GC in GCs
        )
    results = tuple(tuple(i) for i in zip(*results0))
    plot_setup = np.array(results[0:2])
    measures = {'log$_2$(err)':results[2] , 'height difference':results[3]} 
    for measure in measures.keys():
        polar_plot(nrs, GCs, measures[measure], name = measure)
        show_plot(filename = f'convergence_{measure}.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   1 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-2)]: Done  15 out of  25 | elapsed:  2.1min remaining:  1.4min
[Parallel(n_jobs=-2)]: Done  18 out of  25 | elapsed:  2.8min remaining:  1.1min
[Parallel(n_jobs=-2)]: Done  21 out of  25 | elapsed:  3.2min remaining:   36.7s
[Parallel(n_jobs=-2)]: Done  25 out of  25 | elapsed:  3.6min finished
C:\Users\Michael\anaconda3\lib\site-packages\matplotlib\collections.py:857: RuntimeWarning: invalid value encountered in sqrt
  scale = np.sqrt(self._sizes) * dpi / 72.0 * self._factor
C:\Users\Michael\anaconda3\lib\site-packages\matplotlib\projections\polar.py:142: RuntimeWarning: invalid value encountered in remainder
  theta = (np.arctan2(y, x) + 2 * np.pi) % (2 * np.pi)
No handles with labels found to put in leg

log$_2$(err) values (-4.268320705646015, -4.160130640288148, -4.239479676952537, -4.673639640172093, -5.806922877692183, -6.946392915847298, -7.309351694258841, -7.848715450172599, -8.71869633054316, -11.468229551156988, -7.810705704675517, -8.191670843894121, -8.727785037046381, -9.640213564211203, -12.507860650207801, -8.345183853489083, -8.725495578754844, -9.26824458713891, -10.156765120142397, -12.958900419260505, -8.733926513174618, -9.120667062208929, -9.660093415468674, -10.536899613888297, -13.279258885767465)


C:\Users\Michael\anaconda3\lib\site-packages\matplotlib\collections.py:857: RuntimeWarning: invalid value encountered in sqrt
  scale = np.sqrt(self._sizes) * dpi / 72.0 * self._factor
C:\Users\Michael\anaconda3\lib\site-packages\matplotlib\collections.py:857: RuntimeWarning: invalid value encountered in sqrt
  scale = np.sqrt(self._sizes) * dpi / 72.0 * self._factor


No handles with labels found to put in legend.


location values (1.0, 1.0, 1.0, 1.2866476537397291, 1.0, 1.0275086586584865, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0)


...

No handles with labels found to put in legend.


location values (1.0, 1.0, 1.0, 1.2866476537397291, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0)


No handles with labels found to put in legend.


height values (0.928647074464611, 0.8494267783136397, 0.7301001678466569, 0.8508621328299442, 1.0526630546356663, 0.9998540884522794, 1.000223880524431, 1.0004458351402892, 1.0005557513415617, 1.000464897378928, 0.9999654861879828, 1.000062526451991, 1.0001224975547967, 1.0001408631925914, 1.0001194900166792, 0.9999843646578674, 1.0000270328845027, 1.0000536647143417, 1.0000630027283035, 1.0000563189616387, 0.9999911208808248, 1.000015366745517, 1.0000299686237368, 1.000035548550745, 1.000031449472682)


[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   1 tasks      | elapsed:   10.3s
